In [11]:
import csv
import pandas as pd
tmp_catalog = '/home/jimmyli/announcement/'

data_x = pd.read_csv(filepath_or_buffer='/home/jimmyli/announcement/bulletinu.csv', sep = ','
                    )["公告级别"].values
data_y = pd.read_csv(filepath_or_buffer='/home/jimmyli/announcement/bulletinu.csv', sep = ','
                    )["公告主题"].values
Hlength = len(data_x)

f = open('/home/jimmyli/announcement/level_data.txt', 'w')

for i in range(1, Hlength-1):
    # print(data_x[i], data_y[i])
    if (data_x[i] == '中'):
        data_x[i] = '其它'
    elif (data_x[i] == '高'):
        data_x[i] = '重要'
    f.write(str(data_x[i]))
    f.write('\t')
    f.write(str(data_y[i]))
    f.write('\n')
f.close()

In [2]:
import multiprocessing
import jieba

tmp_catalog = '/home/jimmyli/announcement/'

def tokenFile(file_path, write_path):

    with open(write_path, 'w') as w:
        with open(file_path, 'r') as f:
            for line in f:
                token_sen = (" ".join(jieba.cut(line.split('\t')[1])))               
                w.write(line.split('\t')[0] + '\t' + token_sen)
    print(file_path + ' has been token and token_file_name is ' + write_path)

tokenFile(tmp_catalog+'level_data.txt', tmp_catalog+'level_token.txt')


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.491 seconds.
Prefix dict has been built succesfully.


/home/jimmyli/announcement/level_data.txt has been token and token_file_name is /home/jimmyli/announcement/level_token.txt


In [4]:
def splitData(file_path, train_write_path, test_write_path):
    
    with open(train_write_path, 'w') as w:        
        with open(test_write_path, 'w') as t:           
            with open(file_path, 'r') as f:
                flist = f.readlines()
                for i in range(len(flist)):
                    if i % 5 == 0:
                        t.write(flist[i])
                    else:
                        w.write(flist[i])
                    
splitData(tmp_catalog+'level_token.txt', tmp_catalog+'level_train.txt',
         tmp_catalog+'level_test.txt')                

In [5]:
def constructDataset(path):
    """
    path: file path
    rtype: lable_list and corpus_list
    """
    label_list = []
    corpus_list = []
    with open(path, 'r') as p:
        for line in p:
            label_list.append(line.split('\t')[0])
            corpus_list.append(line.split('\t')[1])
    return label_list, corpus_list

file_path = ['level_test.txt', 'level_train.txt']
test_label, test_set = constructDataset(tmp_catalog + file_path[0])   #448
train_label, train_set = constructDataset(tmp_catalog + file_path[1]) #1788

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

corpus_set = train_set + test_set
print("length of corpus is:" + str(len(corpus_set)))

vectorizer = CountVectorizer(min_df=1e-5) # drop df < le-5,去低频词

transfomer = TfidfTransformer()

tfidf = transfomer.fit_transform(vectorizer.fit_transform(corpus_set))

words = vectorizer.get_feature_names()

print("how many words: {0}".format(len(words)))

print("tf-idf shape: ({0}, {1})".format(tfidf.shape[0], tfidf.shape[1]))

length of corpus is:2236
how many words: 2092
tf-idf shape: (2236, 2092)


In [7]:
from sklearn import preprocessing

# encode label
corpus_label = train_label + test_label
encoder = preprocessing.LabelEncoder()
corpus_encode_label = encoder.fit_transform(corpus_label)
train_label = corpus_encode_label[:1788]
test_label = corpus_encode_label[1788:]
# get tf-idf dataset
train_set = tfidf[:1788]
test_set = tfidf[1788:]

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix

# LogisticRegression classiy model
lr_model = LogisticRegression()
lr_model.fit(train_set, train_label)
y_pred = lr_model.predict(test_set)
print (classification_report(test_label, y_pred))

             precision    recall  f1-score   support

          0       0.74      0.95      0.83       274
          1       0.70      0.59      0.64        64
          2       0.62      0.24      0.34       110

avg / total       0.70      0.72      0.68       448



In [10]:
encoder.classes_

array(['其它', '极重要', '重要'], dtype='<U3')